#### I hope every SNS company such as FB, Twitter, and Line implemnt this kind of detection.
This finds the isolated people from the specified community. It is not really called bully, but I named it bully.
The idea is that it tries to find the community with the same member with
one additional member.  And that addional member is bullied by the specified community.
This needs 2 tables.  One table is for Community and another table is for User.
- In the followin case, Community A is bulling to user 1005.
#### Community table: (User 1001, 1002, 1003, 1004, and 1005 are in Community A...)
- A: 1001, 1002, 1003, 1004, 1005
- B: 1001, 1002, 1003, 1004
- C: 1003, 1004, 1006
- D: 10010, 10011
- E: 10010, 10011
- F: 10010, 10011, 10012
#### User table: (User 1001 is in Community A and B...)
- 1001: A, B
- 1002: A, B
- 1003: A, B, C
- 1004: A, B, C
- 1005: A
- 1006: C
- 10010: D E
- 10011: D E
- 10012: D E F

### Load Users from file

In [56]:
""" loading user and community file
format: user communityA communityB
"""

from collections import defaultdict

def read_users(file):
    """ 
    read_users load file and return 2 dictionaries.
    Parameters
    ----------
    file: input file
    format: user communityA communityB ...
    Returns:
    --------
    dictionary:  The key is user, and the value is list of communities.
    dictionary:  The key is community, and the value is list of users.
    
    The following example reads file and create users and communities.
    
    >>> users, communities = read_users('user.lst')
    >>> print(communities['C'])
    [003, 1004, 1006]
    """
    users = defaultdict(list)
    communities = defaultdict(list)
    f = open(file);
    for line in f:
        line = line.rstrip()
        fields = line.split()
        if len(fields) > 1:
            for i in range(1, len(fields)):
                users[int(fields[0])].append(fields[i])
                communities[fields[i]].append(int(fields[0]))
    f.close()
    for key in communities.keys():
        communities[key].sort()
    return users, communities

### Create csr_matrix from the name of community and communities

In [57]:
""" create csr_matrix """
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

def com_matrix(c, communities, shp=None):
    """ com_matrix creates csr_matrix by community name and dictionary of
    communities.
    Parameter
    ---------
    c: community name
    communities: the dictionary of communities
        The key is communitty and the value is the list of users.
    shp: shape of output.
        If shp is None, this uses the shape from community which user belongs to.
    Return
    ------
    csr_matrix
    
    The following example creates csr_matrix.
    
    >>> a = com_matrix('A', communities)
    >>> print(a.todense())
    [[0 0 0 ..., 1 1 1]]
    """
    # c_users: list of users whom c has
    all_users = communities[c]
    # maxsize: the biggest userID
    maxsize = max(all_users) + 1
    
    # prepare data, row, and col
    cdata = np.ones(len(all_users))
    crow = np.zeros(len(all_users))
    ccol = np.zeros(len(all_users))
    for i, user in enumerate(all_users):
        ccol[i] = user

    if shp is not None:
        tshp = shp
    else:
        tshp = (1, maxsize)
        
    com = csr_matrix((cdata, (crow, ccol)), shape=tshp, dtype=np.int64)
    return(com)

In [62]:
""" find similar community and bullied user """

from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

def bully(cmain, communities, users):
    """ Find the community, which has the same users as cmain,
    and it has one more user.
    Parameger
    ---------
    cmain: community
    communities: the dictionary of communities
        The key is communitty and the value is the list of users.
    users: the dictionary of users
        The key is userID and value is the list of communities.
    Return:
    a list of tuple (other community, bullied user)
    
    example:
    >>> a = bully('B', communities, users)
    >>> print(a)
    [('A', 1005)]
    """
    # c_users: list of users whom c has
    c_users = communities[cmain]
    # all_c: all communities which user belongs to.
    all_c = set()
    for user in c_users:
        for c in users[user]:
            all_c.add(c)

    # all_users: all users which all communities have.
    all_users = set()
    # maxsize: the biggest userID + 1
    maxsize = 0
    for v in all_c:
        if maxsize < max(communities[v]):
            maxsize = max(communities[v])
        for u in communities[v]:
            all_users.add(u)
    maxsize += 1
    
    # all_cs: all communities which all associating users belongs to
    all_cs = set()
    all_users_list = list(all_users)
    for user in all_users_list:
        for c in users[user]:
            all_cs.add(c)

    # bully check
    btupple = list()
    cM = com_matrix(cmain, communities, (1, maxsize))
    for c in all_cs:
        cS = com_matrix(c, communities, (1, maxsize))
        cD = cM - cS
#        print(c + ' ' + str(communities[cmain]) + ' ' + str(communities[c]) +
#              ' ' + ' ' + str(cD.nnz))
        if cD.nnz == 1:
            if cM.nnz < cS.nnz:
                btupple.append((c, np.array(cD.argmin(axis=1))[0][0]))
    return(btupple)

### sample test

In [65]:
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

users, communities = read_users('user.lst')
for c in communities:
    bs = bully(c, communities, users)
    if len(bs) > 0:
        for b in bs:
            com, user = b
            print('Community ' + c + ' is bullying ' + str(user))

Community B is bullying 1005


### pgsql solution
This is relatively simpler than code in python, and it only needs community table.